In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
vistas = pd.read_csv('../data/adri_data/entrenamiento_vistas.csv');
vistas.head()

,idaviso,timestamp,idpostulante,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1112400775,2018-04-06T01:43:47.445-0400,kPjr2K0,9,1,20,19427,0,0,0,0,0,182,517
1,1112400775,2018-04-06T01:43:48.330-0400,kPjr2K0,9,1,20,19427,0,0,0,0,0,182,517
2,1112400775,2018-04-06T01:39:36.249-0400,kPjr2K0,9,1,20,19427,0,0,0,0,0,182,517
3,1112400775,2018-04-03T22:41:50.496-0400,rmRpqEA,12,1,32,19427,0,0,0,0,0,182,517
4,1112400775,2018-04-06T01:52:30.411-0400,EzEovD2,4,1,22,19427,0,0,0,0,0,182,517


In [3]:
postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones.csv');
postulaciones.head()

,idaviso,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,fechapostulacion,valor_educacion,sexo,edad
0,1111556097,16272,0,0,0,0,0,50,0,pzeVW63,2018-03-13 16:23:14,17,1,44
1,1112370711,4943,0,0,0,0,0,143,15,pzeVW63,2018-03-08 14:28:36,17,1,44
2,1112326655,18147,0,0,0,0,0,29,25,pzeVW63,2018-02-09 14:35:24,17,1,44
3,1112350182,3957,0,0,0,0,0,119,40,pzeVW63,2018-02-20 15:16:56,17,1,44
4,1112383837,15090,0,0,0,0,0,115,46,pzeVW63,2018-03-07 14:18:18,17,1,44


In [4]:
vistas = vistas[['idaviso',
'idpostulante',
'timestamp',
'valor_educacion',
'sexo',
'edad',
'idtitulo',
'idzona',
'idciudad',
'idmapacalle',
'idtipo_de_trabajo',
'idnivel_laboral',
'idnombre_area',
'iddenominacion_empresa']]

In [5]:
postulaciones = postulaciones[['idaviso',
'idpostulante',
'fechapostulacion',
'valor_educacion',
'sexo',
'edad',
'idtitulo',
'idzona',
'idciudad',
'idmapacalle',
'idtipo_de_trabajo',
'idnivel_laboral',
'idnombre_area',
'iddenominacion_empresa']]

In [6]:
postulaciones.rename({'fechapostulacion': 'timestamp'}, axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1111556097,pzeVW63,2018-03-13 16:23:14,17,1,44,16272,0,0,0,0,0,50,0
1,1112370711,pzeVW63,2018-03-08 14:28:36,17,1,44,4943,0,0,0,0,0,143,15
2,1112326655,pzeVW63,2018-02-09 14:35:24,17,1,44,18147,0,0,0,0,0,29,25
3,1112350182,pzeVW63,2018-02-20 15:16:56,17,1,44,3957,0,0,0,0,0,119,40
4,1112383837,pzeVW63,2018-03-07 14:18:18,17,1,44,15090,0,0,0,0,0,115,46


In [7]:
vistas['se_postulo'] = 0

In [8]:
postulaciones['se_postulo'] = 1

In [9]:
vistas.shape

(9849656, 15)

In [10]:
postulaciones.shape

(5999441, 15)

In [11]:
vistas_y_postulaciones = pd.concat([postulaciones, vistas], axis=0)
vistas_y_postulaciones.shape

(15849097, 15)

In [12]:
vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas.csv', encoding='utf-8')

In [13]:
# Convierto a numerico el id postulante
lb_make = LabelEncoder()
vistas_y_postulaciones["idpostulante2"] = lb_make.fit_transform(vistas_y_postulaciones["idpostulante"])
vistas_y_postulaciones[["idpostulante", "idpostulante2"]].head()

,idpostulante,idpostulante2
0,pzeVW63,365446
1,pzeVW63,365446
2,pzeVW63,365446
3,pzeVW63,365446
4,pzeVW63,365446


In [14]:
postulantes_ids = vistas_y_postulaciones[["idpostulante", "idpostulante2"]].drop_duplicates()
postulantes_ids.to_csv('../data/adri_data/entrenamiento_id_postulantes.csv', encoding='utf-8', index=False)

In [15]:
vistas_y_postulaciones.drop('idpostulante', axis=1, inplace=True)
vistas_y_postulaciones.rename({'idpostulante2': 'idpostulante'}, inplace=True, axis=1)
vistas_y_postulaciones.head()

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,idpostulante
0,1111556097,2018-03-13 16:23:14,17,1,44,16272,0,0,0,0,0,50,0,1,365446
1,1112370711,2018-03-08 14:28:36,17,1,44,4943,0,0,0,0,0,143,15,1,365446
2,1112326655,2018-02-09 14:35:24,17,1,44,18147,0,0,0,0,0,29,25,1,365446
3,1112350182,2018-02-20 15:16:56,17,1,44,3957,0,0,0,0,0,119,40,1,365446
4,1112383837,2018-03-07 14:18:18,17,1,44,15090,0,0,0,0,0,115,46,1,365446


In [16]:
vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8', index=False)

In [31]:
vistas_y_postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8')

In [32]:
vistas_y_postulaciones.shape

(15849097, 15)

In [33]:
vistas_y_postulaciones.head()

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,idpostulante
0,1111556097,2018-03-13 16:23:14,17,1,44,16272,0,0,0,0,0,50,0,1,365446
1,1112370711,2018-03-08 14:28:36,17,1,44,4943,0,0,0,0,0,143,15,1,365446
2,1112326655,2018-02-09 14:35:24,17,1,44,18147,0,0,0,0,0,29,25,1,365446
3,1112350182,2018-02-20 15:16:56,17,1,44,3957,0,0,0,0,0,119,40,1,365446
4,1112383837,2018-03-07 14:18:18,17,1,44,15090,0,0,0,0,0,115,46,1,365446


In [34]:
dup = vistas_y_postulaciones.duplicated(['idaviso', 'idpostulante'])
vistas_y_postulaciones[dup].shape

(6288554, 15)

In [35]:
vistas_y_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], inplace=True)
vistas_y_postulaciones.shape

(9560543, 15)

In [36]:
# vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas3.csv', encoding='utf-8', index=False)

In [37]:
# vistas_y_postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas3.csv', encoding='utf-8')

In [38]:
vistas_y_postulaciones = vistas_y_postulaciones.sample(frac=1).reset_index(drop=True)

In [39]:
# vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas3.csv', encoding='utf-8', index=False)

In [40]:
vistas_y_postulaciones['timestamp'] = vistas_y_postulaciones['timestamp'].apply(pd.to_datetime)

# df = full_data[['idaviso', 'idpostulante', 'timestamp', 'valor_educacion', 'sexo', 'idtitulo', 
#     'idzona', 'idciudad', 'idmapacalle', 'idtipo_de_trabajo',
#     'idnivel_laboral', 'idnombre_area', 'iddenominacion_empresa', 'se_postulo']]
# 
now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)

vistas_y_postulaciones['timestamp'] = (now - vistas_y_postulaciones['timestamp']).apply(lambda x: x.total_seconds())
# vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas4.csv', encoding='utf-8', index=False)

In [41]:
vistas_y_postulaciones.head()

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,idpostulante
0,1112471469,3.993812e+06,11,2,33,1591,0,0,0,0,0,5,961,0,130270
1,1112372006,8.419868e+06,6,2,35,13934,0,0,0,0,0,142,29,1,186313
2,1112366314,6.931198e+06,12,1,43,3242,0,0,0,0,0,45,148,1,314556
3,1112359660,6.493666e+06,11,1,22,9285,0,0,0,0,0,154,670,0,296986
4,1112300321,1.100474e+07,12,1,25,321,0,0,0,0,0,5,22,1,301715


In [42]:
vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas4.csv', encoding='utf-8', index=False)